<a href="https://colab.research.google.com/github/RonSK03/Resnet18/blob/main/Vehicle_Detection_Resnet18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#linking google drive to access the dataset

Mounted at /content/drive


In [ ]:
data_dir='/content/drive/MyDrive/Vehicles/'

In [ ]:
import numpy as np
import pandas as pd
import torch.nn as nn
import torch
import torchvision
import torchvision.models as models
import matplotlib.pyplot as plt
#importing important libraries

In [ ]:
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader
#importing functions for data processing and splitting

In [ ]:
transform=transforms.Compose([transforms.RandomHorizontalFlip(), #lets model know not just one orientation
                              transforms.RandomResizedCrop(224), #standard size for resnet is 224x224
                              transforms.ToTensor(),#converts the images to a tensor
                              transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])

                              ])
#transforms required to convert image into form needed by ResNet 18 (normalize values used are the standard ones for ResNet18)

In [ ]:
net_data=ImageFolder(data_dir,transform=transform)

In [ ]:
train_size = int(0.8 * len(net_data))
test_size = len(net_data) - train_size

train_data, test_data = random_split(net_data, [train_size, test_size])
#splitting dataset as 80% training and 20% testing

In [ ]:
class_names=train_data.dataset.classes

In [ ]:
class_names

['Auto Rickshaws', 'Bikes', 'Cars', 'Motorcycles', 'Planes', 'Ships', 'Trains']

In [ ]:
train_data_loaded=DataLoader(train_data, batch_size=32, shuffle=True)
#using dataloader to split the data into batches

In [ ]:

train_data_loaded

In [ ]:
test_data_loaded=DataLoader(test_data, batch_size=32, shuffle=True)

In [ ]:
model=models.resnet18(weights=True)
#calling our model resnet18

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 184MB/s]


In [ ]:
for name, param in model.named_parameters():
    if "fc" in name:
        param.requires_grad=True
    else:
        param.requires_grad=False
#freezing all layers and using only the final layer

In [ ]:
criterion=nn.CrossEntropyLoss() #loss/cost function checking how bad it is doing
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-2) #gradient descent alpha=lr


In [ ]:
optimizer.zero_grad()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device being used:", device)
#enabling gpu if available

Device being used: cuda


In [ ]:
model=model.to(device)
#sending the model to gpu

In [ ]:
for epochs in range(10): #10 iterations of training
    model.train()
    with torch.set_grad_enabled(True):
        for inputs, labels in train_data_loaded: #seperating the training data into inputs and labels to compare the prediction and the label
          inputs=inputs.to(device) #since model is in gpu we have to send the inputs and labels too
          labels=labels.to(device)
          optimizer.zero_grad()
          outputs=model(inputs)
          _, preds=torch.max(outputs,1) # the output returns a tensor with the scores of each class so here we extract the index of prediction with highest score and its value
          loss=criterion(outputs, labels) #we find the loss
          loss.backward() #backpropagation of loss
          optimizer.step()
print("Training Complete")

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Training Complete


In [ ]:
model.eval()#changes model mode to evaluation
with torch.set_grad_enabled(False):
    running_corrects=0
    for inputs, labels in test_data_loaded:
        inputs=inputs.to(device)
        labels=labels.to(device)
        outputs=model(inputs)
        _, preds=torch.max(outputs, 1)
        loss=criterion(outputs, labels)
        running_corrects += torch.sum(preds == labels.data)
    epoch_acc = running_corrects.double() /len(test_data_loaded.dataset) #formula for calculation of accuracy
    print(f"Accuracy={epoch_acc}")
print("Test complete")
torch.save(model.state_dict(), "/content/drive/MyDrive/vehicle_detection_model.pth") #saving the model to my google drive

Accuracy=0.9490161001788909
Test complete


In [ ]:
from PIL import Image
import cv2
model.load_state_dict(torch.load("/content/drive/MyDrive/vehicle_detection_model.pth")) #loading saved model for using
model.eval()
image_path="/content/drive/MyDrive/Copy of Car (1).jpg"
image=Image.open(image_path)

In [ ]:
image_tensor=transform(image) #converting test image to tensor

In [ ]:
with torch.set_grad_enabled(False):
    image_tensor_unsqueezed=image_tensor.unsqueeze(0)
    image_tensor_gpu=image_tensor_unsqueezed.to(device)
    outputs=model(image_tensor_gpu)
    _, preds=torch.max(outputs, 1)
print("Prediction is ", class_names[preds.item()])

Prediction is  Cars


In [ ]:
def predicter(image):
  transform=transforms.Compose([transforms.RandomHorizontalFlip(),
                              transforms.RandomResizedCrop(224),
                              transforms.ToTensor(),
                              transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
                              ])
  image_tensor=transform(image)
  image_unsqueezed=image_tensor.unsqueeze(0)
  image_gpu=image_unsqueezed.to(device)
  outputs=model(image_gpu)
  _, preds=torch.max(outputs, 1)
  print("Prediction is ", class_names[preds.item()])
  #defining a function to make it easier to skip steps and directly upload the image after using pillow library


In [ ]:
image2=Image.open("/content/drive/MyDrive/Copy of Plane (33).jpg")

In [ ]:
predicter(image2)

Prediction is  Planes
